In [ ]:
from gurobipy import *
import matplotlib.pyplot as pyplot
from math import exp

In [ ]:
m = Model()
v0 = m.addVar()
v1 = m.addVar()
m.update()
m.addConstr(v0-v1 <= 4)
m.addConstr(v0 + v1 <= 4)
m.addConstr(-0.25*v0 + v1 <= 1)
m.setObjective(v1, GRB.MAXIMIZE)
m.params.outputflag = 0
m.optimize()

In [ ]:
pyplot.plot([0,4], [0,4])
pyplot.plot([4,0], [0,4])
pyplot.plot([0,4], [1,2])
pyplot.plot([v0.x], [v1.x], 'ro')
pyplot.show()

# Lineare Programmierung
<br>https://www.youtube.com/watch?v=p_-MQzeFlh0
<br>
<br> Maximieren Sie: Z = x2 + 2 * x1
<br>
<br> Nebenbedingungen:
<br> 1. x1 >= 0
<br> 2. x2 <= 6 + x1
<br> 3. x2 <= 5 + x1 * 1/4
<br> 4. x2 <= 34/3 - x1 * 4/3
<br> 5. x2 >= -4 + x1 * 1/5

In [ ]:
m = Model()
x1 = m.addVar()
x2 = m.addVar(lb=-4)

z = x2 + 2*x1
m.update()

m.addConstr(x1 >= 0)
m.addConstr(x2 <= 6 + x1)
m.addConstr(x2 <= 5 + x1*1/4)
m.addConstr(x2 <= 34/3 - x1*4/3)
m.addConstr(x2 >= -4 + x1*1/5)

m.setObjective(z, GRB.MAXIMIZE)
m.optimize()

In [ ]:
pyplot.plot([0,6], [6,13])
pyplot.plot([0,16], [5,9])
pyplot.plot([0,10], [11,-2])
pyplot.plot([0,20], [-4,0])
pyplot.plot([x1.x], [x2.x], 'ro')
pyplot.xlabel('x1')
pyplot.ylabel('x2')
pyplot.show()

# Piecewise Linear Objective
https://www.gurobi.com/documentation/8.1/examples/piecewise_py.html#subsubsection:piecewise.py

In [ ]:
def f(u):
    return exp(-u)

def g(u):
    return 2 * u * u - 4 * u

try:

    # Create a new model
    m = Model()

    # Create variables
    lb = 0.0
    ub = 1.0

    x = m.addVar(lb, ub, name='x')
    y = m.addVar(lb, ub, name='y')
    z = m.addVar(lb, ub, name='z')

    # Set objective for y
    m.setObjective(-y)

    # Add piecewise-linear objective functions for x and z
    npts = 101
    ptu = []
    ptf = []
    ptg = []

    for i in range(npts):
        ptu.append(lb + (ub - lb) * i / (npts - 1))
        ptf.append(f(ptu[i]))
        ptg.append(g(ptu[i]))

        
    #print('ptu: ', type(ptu))
    #print('ptu: ', ptu)
    #print('ptf: ', type(ptf))
    m.setPWLObj(x, ptu, ptf)
    m.setPWLObj(z, ptu, ptg)
    
    m.update()

    #print('x: ', m.getPWLObj(x))
    #print('z: ', m.getPWLObj(z))

    # Add constraint: x + 2 y + 3 z <= 4
    m.addConstr(x + 2 * y + 3 * z <= 4, 'c0')

    # Add constraint: x + y >= 1
    m.addConstr(x + y >= 1, 'c1')

    # Optimize model as an LP
    m.optimize()
    
    #print('bla:', m.getPWLObj(x))

    print('IsMIP: %d' % m.IsMIP)
    for v in m.getVars():
        print('%s %g' % (v.VarName, v.X))
    print('Obj: %g' % m.ObjVal)
    print('')
    

    # Negate piecewise-linear objective function for x
    for i in range(npts):
        ptf[i] = -ptf[i]

    m.setPWLObj(x, ptu, ptf)

    # Optimize model as a MIP
    m.optimize()

    print('IsMIP: %d' % m.IsMIP)
    for v in m.getVars():
        print('%s %g' % (v.VarName, v.X))
    print('Obj: %g' % m.ObjVal)

except GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Encountered an attribute error')
    

# Simple Audio Example

In [ ]:
mischsignal = [0.0, 1.0, 1.0, 3.0, 2.0, 0.0]
ton_1 = [0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
ton_2 = [0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
ton_3 = [0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
ton_4 = [0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
ton_5 = [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
freq = [0.0, 1.0, 2.0, 3.0, 4.0, 5.0]

In [ ]:
pyplot.plot(freq, ton_1)
pyplot.plot(freq, ton_2)
pyplot.plot(freq, ton_3)
pyplot.plot(freq, ton_4)
pyplot.plot(freq, ton_5)
pyplot.plot(freq, mischsignal)
pyplot.xlabel('Frequenz')
pyplot.ylabel('Amplitude')
pyplot.show()

In [ ]:
m = Model()
lamb_1 = m.addVar(vtype = GRB.INTEGER, name = 'lamb_1')
#lamb_2 = m.addVar(vtype = GRB.INTEGER, name = 'lamb_2')
#lamb_3 = m.addVar(vtype = GRB.INTEGER, name = 'lamb_3')
#lamb_4 = m.addVar(vtype = GRB.INTEGER, name = 'lamb_4')

m.setObjective(lamb_1, GRB.MAXIMIZE)
#m.setObjectiveN(lamb_1, 1) # Position ist wichitg!!!
#m.setObjectiveN(lamb_2, 2)
#m.setObjectiveN(lamb_3, 3)
#m.setObjectiveN(lamb_4, 4)
m.update()

t1 = m.addVar(name = 't1')
t2 = m.addVar(name = 't2')
t3 = m.addVar(name = 't3')
t4 = m.addVar(name = 't4')
mix = m.addVar(name = 'mix')

m.setPWLObj(t1, freq, ton_1)
#m.setPWLObj(t2, freq, ton_2)
#m.setPWLObj(t3, freq, ton_3)
#m.setPWLObj(t4, freq, ton_4)
m.setPWLObj(mix, freq, mischsignal)

m.update()

#m.addConstr(mix - (lamb_1 * t1 +
#                   lamb_2 * t2 +
#                   lamb_3 * t3 +
#                   lamb_4 * t4)<= 1)

print('mix:', m.getPWLObj(mix))
print('t1', m.getPWLObj(t1))

#m.addConstr(mix - (lamb_1*t1) <= 10)
#m.addConstr(lamb_1 <= 10)
#m.addConstr(2 * lamb_1 + 3 * t1 + mix <= 10, 'c0')
#m.addConstr(lamb_1 * t1 <= 3)
m.addConstr(mix >= lamb_1*t1)

m.optimize()

print('t1: ', m.getPWLObj(t1))
print('mix: ', m.getPWLObj(mix))

for v in m.getVars():
    print('%s: %g' % (v.varName, v.x))